In [128]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt
from scipy.linalg import hadamard
import statistics
from numpy.linalg import norm


In [2]:
num_user=[1000, 10000, 100000, 1000000]
all_d = [32, 64, 128, 256, 512]
num_repeats = 20

In [3]:
def naive_sum(I, phi, tau):
    
    norm=np.linalg.norm(I, ord=2, axis=1, keepdims=False)
    trunc_vector = tau/norm
    trunc_vector[trunc_vector>1] = 1
    trunc_vector = np.reshape(trunc_vector, [-1,1])
    I_truc = np.multiply(I, trunc_vector)
    trunc_sum = np.sum(I_truc, axis=0, keepdims=False)
    noise_scale  = np.sqrt(np.sum(1/(2*phi)))
    noisy_sum = trunc_sum + np.random.normal(0, noise_scale, size = d)*tau
#     acc = np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm)
#     print(acc)
    return noisy_sum

In [150]:
#different users
times = []
relative_error = []
real_results = []
for i in range(len(num_user)):
    n = num_user[i]
    B = 1000000
    d = 128
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh)
    
    #Gaussian privacy
#     phi = np.random.normal(10, 3,int(n))
#     phi[phi<1/(10*n)]=1/(10*n)

    #Uniform privacy 
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)
    
    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
    I[I<0]=1
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

    
  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = naive_sum(I, phi, tau = 500*math.sqrt(d))
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)
    real_results.append(l2_norm)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)

print("relative_error")
print(relative_error)
    

times
[0.0004961490631103516, 0.0025200843811035156, 0.01917886734008789, 0.21416997909545898]
relative_error
[0.4157328211307798, 0.4427568468083193, 0.43089007636778887, 0.4809462727625264]


In [151]:
def radius_sum(I, phi, B, beta, d):
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    noisy_sum = np.zeros([t,d])
    each_tau = np.zeros(t)
    cur_min = 100
    for i in range(t):
        tau_i = math.pow(2,i)*np.sqrt(phi/phi_max)
        norm=np.linalg.norm(I, ord=2, axis=1, keepdims=False)
        trunc_vector = tau_i/norm
        trunc_vector[trunc_vector>1] = 1
        trunc_vector = np.reshape(trunc_vector, [-1,1])
        I_truc = np.multiply(I, trunc_vector)
        trunc_sum = np.sum(I_truc, axis=0, keepdims=False)
        noisy_sum[i] = trunc_sum + np.random.normal(0, math.sqrt(n*math.pow(2,2*i)*t/(2*phi_max)), size = d) - math.pow(2,i)*math.sqrt(n*t/(2*phi_max)*math.log(t*d/beta))
        noisy_sum[i][noisy_sum[i]<0]=0
        #l2 error
        each_tau[i] =np.sqrt(np.sum(((true_sum-noisy_sum[i]))**2))/(l2_norm)
        if each_tau[i]<cur_min:
            idx = i
            cur_min = each_tau[i]


    best_sum = noisy_sum[idx]+math.pow(2,idx)*math.sqrt(n*t/(2*phi_max)*math.log(t*d/beta))

#     best_sum = noisy_sum.max(axis=0)
#     best_sum[best_sum<0] = 0
    best_acc = np.sqrt(np.sum(((true_sum-best_sum))**2))/(l2_norm)
#     print(best_acc)

    return best_sum


In [160]:
#different users
times = []
relative_error = []
real_results = []
for i in range(len(num_user)):
    n = num_user[i]
    B = 1000000
    d = 128
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh) 
    
    #Gaussian privacy
#     phi = np.random.normal(10,3,int(n))
#     phi[phi<1/(10*n)]=1/(10*n)
        

    #Uniform privacy
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)
    
    
    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
    I[I<0]= -I[I<0]
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = radius_sum(I, phi, B, beta, d)
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)

    real_results.append(l2_norm)

    
print("times")
print(times)

print("relative_error")
print(relative_error)

times
[0.019597291946411133, 0.10815000534057617, 0.9146592617034912, 13.207149028778076]
relative_error
[0.2270987733627783, 0.21287221983004578, 0.0738825104856175, 0.0598727794066842]


In [153]:
def random_hadamard(d,t=1):
    H = hadamard(d)
    #print(np.ones((d,1))))
    #print(np.matmul(H,np.transpose(H)))
    rand = np.random.uniform(low=0,high=1,size=d)
    flipped = (rand < 0.5)
    diagonal = np.ones(d)
    diagonal[flipped] = -1
    D = np.diag(diagonal)
    HD = np.matmul(H,D)
    # rotate t-1 more times
    for i in range(t-1):
        rand = np.random.uniform(low=0,high=1,size=d)
        flipped = (rand < 0.5)
        diagonal = np.ones(d)
        diagonal[flipped] = -1
        D = np.diag(diagonal)
        #print(D)
        HD = np.matmul(HD,np.matmul(H,D))
    #print(HD)
    #print(np.matmul(HD,np.transpose(HD)))
    return HD  

In [154]:
def median_selection(column, phi, B, beta):
    T = int(math.log(B, 2))+2
    trimmed = column[nl:nh]
    left = -B
    right = B
    m = nh/2
    i = 0
    while i < T:
        mid = (left+right)/2
        count = len([xi for xi in column if xi<=mid])
        count_noisy = count+ np.random.normal(0, np.sqrt(nh*T/(2*10)))
        if count_noisy <= m:
            left = mid + 1
        else:
            right = mid
        i = i+1
    return int((left+right)/2)

In [157]:
def diam_sum(I, phi, B, beta, d):
    n = len(phi)
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    HD = random_hadamard(d)
    HD_inv = np.transpose(HD)
    I_t = np.transpose(I)
    rotat_I_t = np.matmul(HD, I_t)
    rotat_I = np.transpose(rotat_I_t)
    #find median
    m = np.zeros(d)
    for i in range(d):
        m[i] = median_selection(rotat_I[ : ,i], phi/(2*d), d*B, beta/(4*d))
        
#     print(np.matmul(HD_inv,np.transpose(m) )/d)
    #shift and divide into two parts
    shift_rotate_I = rotat_I-m
    
    xx = np.matmul(HD_inv,np.transpose(shift_rotate_I) )/d
    xx = np.transpose(xx)
    print(norm(xx[0]))
    
    positive = copy.deepcopy(shift_rotate_I)
    negative = -copy.deepcopy(shift_rotate_I)
    positive[positive < 0] = 1e-7
    negative[negative < 0] = 1e-7
    pos_sum = radius_sum(positive, phi/4, B*d, beta/4, d)
    neg_sum = radius_sum(negative, phi/4, B*d, beta/4, d)
    
    rotat_res = pos_sum-neg_sum+m*n
    res = np.matmul(HD_inv,np.transpose(rotat_res) )/d
    res[res<0] = 0
    
    pp_sum = np.sum(positive,axis=0, keepdims=False)
    rr = np.matmul(HD_inv,np.transpose(pp_sum) )/d
#     print((l2_norm)/np.sqrt(np.sum(rr**2)))
    
    return res

In [159]:
#different users
times = []
relative_error = []
real_results = []
for i in range(1):
    n = num_user[i]
    B = 1000000
    d = 128
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh) 
    
    
#     #Gaussian privacy
#     phi = np.random.normal(10,3,int(n))
#     phi[phi<1/(10*n)]=1/(10*n)
    
    #Uniform privacy
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)

    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
#     print(I)
    I[I<0]=1
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    print(norm(I[0]))
  
    start = time.time()
    err = 0
    for j in range(num_repeats):
        noisy_sum = diam_sum(I, phi, B, beta, d)
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)

    real_results.append(l2_norm)

    
print("times")
print(times)

print("relative_error")
print(relative_error)

6726.0522
3238.3092123340384
times
[0.3538529872894287]
relative_error
[0.2507890966896696]


In [53]:
import csv
label = np.zeros(60000)
pixels =np.zeros((60000, 784))
all_I = [[] for _ in range(10)]
i=0
with open('./mnist_train.csv', 'r') as csv_file:
    for data in csv.reader(csv_file):
        # The first column is the label
        label[i] = data[0]
        all_I[int(data[0])].append(data[1:])

        # The rest of columns are pixels
        pixels[i] = data[1:]
        i+=1

label2 = np.zeros(10000)
pixels2 =np.zeros((10000, 784))
i=0
with open('./mnist_test.csv', 'r') as csv_file:
    for data in csv.reader(csv_file):
        # The first column is the label
        label2[i] = data[0]
        all_I[int(data[0])].append(data[1:])

        # The rest of columns are pixels
        pixels2[i] = data[1:]
        i+=1

In [180]:
#digit 0
temp = all_I[1]
# temp = temp+temp+temp
I = np.array(temp,dtype=np.float32)
n, d = I.shape
B = 255*math.sqrt(d)
beta = 0.1

fl = 0.05
fh = 0.95

nl = int(n*fl)
nh = n-nl
phi_l = np.random.uniform(1/(10*n), 100/n, nl)
phi_h = np.random.uniform(10, 100, nh)
phi = np.append(phi_l, phi_h)
phi_max = max(phi)
phi_min = min(phi)
    
t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
true_sum = np.sum(I, axis=0, keepdims=False)
l2_norm = np.sqrt(np.sum(true_sum**2))

In [76]:
#different digits
times = []
relative_error = []
real_results = []
for i in range(10):
    #digit i
    temp = all_I[i]
    temp = temp+temp+temp
    I = np.array(temp,dtype=np.float32)
    n, d = I.shape
    B = 255*math.sqrt(d)
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh) 
    
    
#     #Gaussian privacy
    phi = np.random.normal(10,2,int(n))
    phi[phi<1/(10*n)]=1/(10*n)
    
    #Uniform privacy
#     phi_l = np.random.uniform(1/(10*n), 100/n, nl)
#     phi_h = np.random.uniform(10, 100, nh)
#     phi = np.append(phi_l, phi_h)

    phi_max = max(phi)
    phi_min = min(phi)

    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

    
  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = naive_sum(I, phi, tau = 25*math.sqrt(d))
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)
    real_results.append(l2_norm)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)

print("relative_error")
print(relative_error)
    

times
[0.0198972225189209, 0.021670103073120117, 0.018949031829833984, 0.019707202911376953, 0.018085002899169922, 0.015959978103637695, 0.017534971237182617, 0.01773381233215332, 0.016762733459472656, 0.01671004295349121]
relative_error
[0.7491362905186442, 0.6145996661265909, 0.7278964971912647, 0.7188142817757187, 0.7252000952264801, 0.7067638658746583, 0.7158571912066086, 0.686984572173551, 0.7269465329935549, 0.6965506799001073]


In [78]:
#different digits
times = []
relative_error = []
real_results = []
for i in range(10):
    #digit i
    temp = all_I[i]
    temp = temp+temp+temp

    # temp = temp+temp+temp
    I = np.array(temp,dtype=np.float32)
    n, d = I.shape
    B = 255*math.sqrt(d)
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh) 
    
    
#     #Gaussian privacy
    phi = np.random.normal(10,2,int(n))
    phi[phi<1/(10*n)]=1/(10*n)
    
    #Uniform privacy
#     phi_l = np.random.uniform(1/(10*n), 100/n, nl)
#     phi_h = np.random.uniform(10, 100, nh)
#     phi = np.append(phi_l, phi_h)
    
    phi_max = max(phi)
    phi_min = min(phi)

    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

    
  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = radius_sum(I, phi, B, beta, d)
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)
    real_results.append(l2_norm)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)

print("relative_error")
print(relative_error)

times
[0.565117359161377, 0.8768820762634277, 0.5828330516815186, 0.7264549732208252, 0.5473248958587646, 0.5288488864898682, 0.5589010715484619, 0.7889389991760254, 0.5580489635467529, 0.5668671131134033]
relative_error
[0.766948688102325, 0.6323192511731122, 0.86453040124817, 0.8776173103001922, 1.0076812598890754, 1.0703402491869491, 0.9298743140453106, 1.0387577334212958, 0.8688987692437807, 1.0045128465782902]


In [142]:
#different digits
times = []
relative_error = []
real_results = []
for i in range(10):
    #digit i
    temp = all_I[i]
    temp = temp+temp+temp
    I = np.array(temp,dtype=np.float32)
    n, _ = I.shape
    I_pad = np.zeros((n, 1024))
    I_pad[:, 0:784] = I
    I_pad[I_pad==0] =   1e-7
    n, d = I_pad.shape
    
    B = 255*math.sqrt(784)
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh) 
    
    
#     #Gaussian privacy
    phi = np.random.normal(10,2,int(n))
    phi[phi<1/(10*n)]=1/(10*n)
    
    #Uniform privacy
#     phi_l = np.random.uniform(1/(10*n), 100/n, nl)
#     phi_h = np.random.uniform(10, 100, nh)
#     phi = np.append(phi_l, phi_h)

    phi_max = max(phi)
    phi_min = min(phi)

    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    print(l2_norm)
    err = 0

    
  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = diam_sum(I_pad, phi, B, beta, d)
        noisy_sum = noisy_sum[0:784]
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)
    real_results.append(l2_norm)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)

print("relative_error")
print(relative_error)



43686572.0
32425704.0
37879136.0
38778310.0
33116080.0
29795124.0
37158190.0
35130170.0
38858316.0
35135050.0
times
[23.532707929611206, 27.384523153305054, 24.4876389503479, 24.767340898513794, 23.822087049484253, 22.09067702293396, 23.74393081665039, 25.23773193359375, 24.384086847305298, 24.553465127944946]
relative_error
[0.04024637787232119, 0.0882339991005018, 1.243895078078368, 0.07012491906771287, 0.04301050671747942, 0.4573734904813865, 0.4898680342187033, 0.05537542018613129, 0.06727268069949752, 0.05266946317237422]


In [75]:
n=1000
fl = 0.05
fh = 0.95
nl = int(n*fl)
nh = int(n*fh) 
phi = np.random.normal(10,3,int(n))
phi[phi<1/(10*n)]=1/(10*n)

#Uniform privacy
# phi_l = np.random.uniform(1/(10*n), 100/n, nl)
# phi_h = np.random.uniform(10, 100, nh)
# phi = np.append(phi_l, phi_h)

phi.sort()

print(phi[1:1000])

[ 0.80763834  1.53843204  1.63740392  2.00898554  2.15332528  2.15801205
  2.22807189  2.28678601  2.4255779   2.52434533  2.57530794  2.67241962
  2.90958636  3.08745067  3.13135571  3.66409143  3.71578539  3.7490093
  3.78016068  3.79953511  3.83163279  3.84061324  3.88356601  3.93756343
  4.0004003   4.03088481  4.09820369  4.12151344  4.1987057   4.27330415
  4.28082099  4.36808173  4.46191668  4.53991903  4.55496102  4.59864049
  4.61559816  4.61623285  4.62471713  4.63263764  4.65000051  4.6977275
  4.8207543   4.88809119  5.00383361  5.03382742  5.0340491   5.03617976
  5.05229354  5.05485609  5.07292438  5.08202213  5.09173797  5.1027927
  5.17559991  5.25437505  5.25505093  5.3207251   5.32421433  5.3930825
  5.3946939   5.43254528  5.43634355  5.46682872  5.49063767  5.50569497
  5.57542848  5.57853989  5.59167322  5.6366379   5.64633429  5.64992766
  5.67177452  5.70052128  5.70418632  5.73816014  5.7400801   5.78314552
  5.78847452  5.81199707  5.82767405  5.83362219  5.840